# 

# Stochastic Volatility Model on Returns

**Model Specification**

*De-Meaned Returns $y_t$:*

$$
y_t = e^{\frac{h_t}{2}} \cdot \epsilon_t
$$

*Log-Volatility $h_t$:*
$$ 
h_t = \mu + \phi(h_{t-1} - \mu) + \sigma \cdot \eta_t
$$


$\epsilon_t$ and $\eta_t$ are gaussian white noise:

$$
\epsilon_t \sim \mathcal{N}(0,1)
$$
$$
\eta_t \sim \mathcal{N}(0,1)
$$

**Parameters**

$\phi \in (-1, 1)$

$ \sigma > 0 $


**Priors**

$\mathcal{P}(\mu) \sim \mathcal{N}(...)$

$\mathcal{P}(\phi) \sim \mathcal{N}(...) \cdot \mathbb{1}_{[-1, 1]}$ 

$\mathcal{P}(\sigma) \sim IG(...)$




# GARCH(1,1) Model 

**Model Specification**

*De-Meaned Returns $y_t$:*

$$
y_t = \epsilon_t
$$

*Conditional Variance Equation (GARCH(1,1)):*
$$
\sigma_t^2 = \mu + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$


$\epsilon_t$ is a gaussian error term:

$$
\epsilon_t \sim \mathcal{N}(0,\sigma_t^2)
$$


**Parameters:**
- $\mu > 0$ ensures non-negative variance.
- $\alpha_1, \beta_1 \geq 0$ and $\alpha_1 + \beta_1 < 1$ for stationarity.


**Priors**

$\mu \sim IG(...)$

$\alpha_1 \sim \beta(...)$ 

$\beta_1 \sim \beta(...)$






# DCC GARCH Model

**Model Specification**

Each time series $ r_{it} $ for asset $ i $ and time $ t $ is modeled separately using a GARCH(1,1) model:
- Equation for returns:
  $$
  r_{it} = \mu_i + \epsilon_{it}
  $$
- Equation for residuals:
  $$
  \epsilon_{it} = H_t^{\frac{1}{2}} z_{it}
  $$
$$
  \text{Where}: H_t = D_tR_tD_t
$$



<br>
**Notation and Definitions**

- **$r_t$:** 
  - $n \times 1$ vector of log returns of $n$ assets at time $t$.

- **$\epsilon_t$:** 
  - $n \times 1$ vector of mean-corrected returns of $n$ assets at time $t$, where $E[\epsilon_t] = 0$ and $\text{Cov}[\epsilon_t] = H_t$.

- **$\mu_t$:** 
  - $n \times 1$ vector representing the expected value of the conditional returns at time $t$.

- **$H_t$:** 
  - $n \times n$ matrix of conditional variances of $\epsilon_t$ at time $t$.

- **$H_t^{1/2}$:** 
  - Any $n \times n$ matrix at time $t$ such that $H_t$ is the conditional variance matrix of $\epsilon_t$. Can be obtained by a Cholesky factorization of $H_t$.

- **$D_t$:**
  - $n \times n$ diagonal matrix of conditional standard deviations of $\epsilon_t$ at time $t$.

- **$R_t$:**
  - $n \times n$ conditional correlation matrix of $\epsilon_t$ at time $t$.

- **$z_t$:**
  - $n \times 1$ vector of iid errors such that $E[z_t] = 0$ and $E[z_t z_t^T] = I$.




<br>

**Stage 1: Univariate GARCH Models**

The elements of the diagonal matrix $D_t$ are obtained by estimating unvariate GARCH models for the time series of each asset:

  $$
  D_t = \begin{bmatrix}
  \sqrt{h_{1t}} & 0 & \cdots & 0 \\
  0 & \sqrt{h_{2t}} & \cdots & 0 \\
  \vdots & \vdots & \ddots & \vdots \\
  0 & 0 & \cdots & \sqrt{h_{nt}}
  \end{bmatrix}
  $$

  where each variance component $h_{it}$ is modeled as:
  $$
  h_{it} = \alpha_{i0} + \sum_{q=1}^{Q_i} \alpha_{iq} \epsilon_{i, t-q}^2 + \sum_{p=1}^{P_i} \beta_{ip} h_{i, t-p}
  $$


<br>

**Stage 2: Dynamic Conditional Correlations (DCC)**

The DCC part of the model captures the time-varying correlations between the assets:
- DCC equation for the covariance matrix:
  $$
  \mathbf{Q}_t = \overline{\mathbf{Q}} (1 - a - b) + a (\mathbf{z}_{t-1} \mathbf{z}_{t-1}') + b \mathbf{Q}_{t-1}
  $$
- Normalizing $ \mathbf{Q}_t $ to obtain the correlation matrix:
  $$
  \mathbf{R}_t = \text{diag}(\mathbf{Q}_t)^{-1/2} \mathbf{Q}_t \text{diag}(\mathbf{Q}_t)^{-1/2}
  $$
where:
- $ \mathbf{Q}_t $ is the matrix of conditional covariances of the standardized residuals,
- $ \overline{\mathbf{Q}} $ is the unconditional covariance matrix of the standardized residuals $ \mathbf{z}_{t} $,
- $ \mathbf{R}_t $ is the matrix of conditional correlations,
- $ a $ and $ b $ are non-negative parameters satisfying $ a + b < 1 $ for stationarity.



<br>

**Model Parameters**


- **$\mu_i$:** 
  - Mean of asset returns

- **$\alpha_{i0}$, $\alpha_{iq}$, $\beta_{ip}$:**
  - GARCH parameters for the variance equation of each asset.
  - $\alpha_{iq}$ + $\beta_{ip}$ < 1 for each $i$ (for stationarity)

- **$a$, $b$:**
  - Parameters of the DCC model controlling the dynamics of the correlation updates
  - $a, b > 0$ and $a + b < 1$ for stationarity.


**Priors**

*Mean of Asset Returns ($\mu_i$)*

- **Normal Prior**: 
  $$
  \mu_i \sim \text{Normal}(0, 1^2)
  $$

*GARCH Parameters ($\alpha_{i0}, \alpha_{iq}, \beta_{ip}$)*
- **Gamma Prior for $\alpha_{i0}$**:
  $$
  \alpha_{i0} \sim \text{Gamma}(...)
  $$
- **Beta Priors for $\alpha_{iq}$ and $\beta_{ip}$**: 
  $$
  \alpha_{iq}, \beta_{ip} \sim \text{Beta}(...)
  $$

*DCC Parameters ($a, b$)*
- **Beta Priors**: 
  $$
  a, b \sim \text{Beta}(...)
  $$





# Latent Factors with GARCH

**Model Overview**


$
y_t = \mu + \epsilon_t
$
   
Here, $y_t$ is the observed vector at time $t$, $\mu$ is a constant vector, and $\epsilon_t$ is the innovation vector at time $t$.

<br>

$
\epsilon_t = W X_t
$
   
$W$ is a parameter matrix, and $X_t$ is a vector of latent factors.

$
X_t | \Theta_{t-1} \sim \mathcal{N}(0, \Sigma_t)
$

$\Theta_{t-1}$ represents the information set up to time $t-1$, and $X_t$ is multivariate normal given $\Theta_{t-1}$ with mean zero and covariance matrix $\Sigma_t$.


<br>

$
\Sigma_t = \text{diag}(\sigma^2_{1,t}, \ldots, \sigma^2_{N,t})
$

<br>
<br>

**Each $\sigma^2_{i,t}$ follows a GARCH(1,1) process**:
<br>

$
\sigma^2_{i,t} = \alpha_i + b_i x^2_{i,t-1} + g_i \sigma^2_{i,t-1}
$
<br>

Where $\alpha_i > 0$, $b_i \geq 0$, and $g_i \geq 0$.


$
\epsilon_t | \Theta_{t-1} \sim \mathcal{N}(0, H_t)
$

$H_t$ is the covariance matrix of $\epsilon_t$ given $\Theta_{t-1}$.


$
H_t = W \Sigma_t W^T = (W \Sigma_t^{1/2}) (W \Sigma_t^{1/2})^T = L L^T
$

